## File names here:

In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']
tss_annotation = doc['tss_annotation']

In [2]:
from create_output import *

txn_f = "Results/tss_annotation/txn_df_02_tissues.p"
#expr_f = "Results/merged/samples.merge.peaksexpression.log10"
expr_f = "Results/merged/tissues.merge.peaksexpression.log10"
anno_f = "Results/tss_annotation_peaks/all_peaks_txn_df_02_maxinfo.tsv"
rank_func = rank_median_samples
atac_f = "../10_15_Results/Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort"
tss_f = "/data/isshamie/genome/picr_final/mRNA_final.peak"#tss_annotation
all_atac = "../10_15_Results/Results/ATAC_results/ATAC_merge.bed"



In [3]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [4]:
if not os.path.exists("Results/output"):
    os.mkdir("Results/output")

In [5]:
%%time
# bed_df, meta_df = construct_peakID(txn_anno_f, expr_f, anno_f, rank_func,
#                      atac_f=None, col_name=None)

bed_df, meta_df = construct_all_peaks(txn_f, expr_f, anno_f, rank_func, tss_f, atac_f,
                        all_atac=all_atac, out_f="Results/output/TSS1")

Constructing TSS not observed experimentally


29749it [17:06, 18.04it/s]


Constructing TSS observed experimentally


0it [00:00, ?it/s]

Including ATAC information


61309it [07:31, 135.79it/s]
61309it [17:48, 57.38it/s]


CPU times: user 56min, sys: 25.7 s, total: 56min 26s
Wall time: 56min 3s


In [6]:
bed_df.head()

,Chr,Start,End,ID,Stat,Strand
p0@SERHL_asmbl_2,picr_0,175,177,p0@SERHL_asmbl_2,0.00000,+
p0@SERHL_asmbl_11,picr_0,18148,18150,p0@SERHL_asmbl_11,0.00000,+
p0@SERHL_asmbl_12,picr_0,20786,20788,p0@SERHL_asmbl_12,0.00000,+
p1@SERHL_asmbl_14,picr_0,20916,21067,p1@SERHL_asmbl_14,2.05576,+
p0@SERHL_asmbl_15,picr_0,22503,22505,p0@SERHL_asmbl_15,0.00000,+


In [7]:
meta_df

,Tissues,cs,CHO ATAC Region,ID,Gene,Gene ID,Transcript,Is Experimental,has ATAC
p0@SERHL_asmbl_2,,,,,SERHL,,asmbl_2,0,0
p0@SERHL_asmbl_11,,,,,SERHL,,asmbl_11,0,0
p0@SERHL_asmbl_12,,,,,SERHL,,asmbl_12,0,0
p1@SERHL_asmbl_14,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,,160866,SERHL,,asmbl_14,1,0
p0@SERHL_asmbl_15,,,,,SERHL,,asmbl_15,0,0
p1@RRP7A_asmbl_10,"Heart,Spleen,MiscOrgans,FemaleReproductive,CHO...",0b11,,298180,RRP7A,,asmbl_10,1,0
p0@RRP7A_asmbl_16,,,,,RRP7A,,asmbl_16,0,0
p0@RRP7A_asmbl_9,,,,,RRP7A,,asmbl_9,0,0
p0@RRP7A_asmbl_7,,,,,RRP7A,,asmbl_7,0,0
p0@RRP7A_asmbl_8,,,,,RRP7A,,asmbl_8,0,0


-----------------------------